# Capstone Project - Hypothetical Neighborhood Survery for B2B 
## Brief Description: This project seeks to identify places where a proposed IT managed service provider (The Company) wants to offer b2b IT systems, networks, and data management solutions to small to medium sized businesses.  

In [1]:
#libraries installed
!pip install geopy
!pip install pandas 
!pip install beautifulsoup4
!pip install requests
!pip install kmeans
!pip install folium
!pip install -U scikit-learn
!pip install soupsieve
!pip install lxml
!pip install matplotlib
!pip install json_flatten
print ("libraries installed")

  Using cached kmeans-1.0.2.tar.gz (5.9 kB)
  Running setup.py clean for kmeans
Failed to build kmeans
    Running setup.py install for kmeans: started
    Running setup.py install for kmeans: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ahorv\\AppData\\Local\\Temp\\pip-install-nvi1llks\\kmeans_3f144a7433444f5790bf312b31564a4b\\setup.py'"'"'; __file__='"'"'C:\\Users\\ahorv\\AppData\\Local\\Temp\\pip-install-nvi1llks\\kmeans_3f144a7433444f5790bf312b31564a4b\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ahorv\AppData\Local\Temp\pip-wheel-wl87fl8n'
       cwd: C:\Users\ahorv\AppData\Local\Temp\pip-install-nvi1llks\kmeans_3f144a7433444f5790bf312b31564a4b\
  Complete output (19 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\kmeans
  copying kmeans\performance.py -> build\lib.win-amd64-3.8\kmeans
  copy

  Using cached scikit_learn-1.0-cp38-cp38-win_amd64.whl (7.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\scikit_learn-0.24.1.dist-info\\COPYING'
Consider using the `--user` option or check the permissions.



libraries installed


In [2]:
# import all necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup #scrape web page
import requests # library to handle requests
#from flatten_json import flatten # flatten nested json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from folium.plugins import MarkerCluster


print('Imports complete.')

Imports complete.


In [3]:
# Using bs4 with CSS selectors to find and scape data from table on web

url1 = "https://en.wikipedia.org/wiki/List_of_highest-income_ZIP_Code_Tabulation_Areas_in_the_United_States"
soup = BeautifulSoup(requests.get(url1).content, "html.parser")

tbl = soup.select("h2:has(#ZCTAs_ranked_by_per_capita_income) + table")
df_zip_income = pd.read_html(str(tbl))[0]
print("df_zip_income created")

df_zip_income created


In [4]:
#organize column headers
df_zip_income.columns = ['rank', 'place', 'zip', 'population', 'per_cap_inc']
df_zip_income.head()

,rank,place,zip,population,per_cap_inc
0,1,"Montchanin, Delaware[2]",19710,68,654485
1,2,"Houston, Texas",77010,76,283189
2,3,"Rockland, Delaware[3]",19732,77,279424
3,4,"Miami Beach, Florida",33109,467,236238
4,5,"Pineland, Florida",33945,79,162075


In [5]:
#load zipcode csv from github into dataframe
url2 = "https://raw.githubusercontent.com/adhorvitz/IBM_Coursera_Capstone/main/uszips_mod_zipgeoonly.csv"
df_zip_geo = pd.read_csv(url2, index_col=0)
df_zip_geo.head()

,lat,lng
zip,,
601,18.18005,-66.75218
602,18.36074,-67.17519
603,18.45440,-67.12201
606,18.16721,-66.93828
610,18.29032,-67.12244


In [6]:
#merge dataframes together
df_zip_good = pd.merge(df_zip_geo, df_zip_income, how = 'left', left_on = 'zip', right_on = 'zip')
print (df_zip_good.shape)

(33121, 7)


In [7]:
#sort values
df_zip_good.sort_values(by=['rank'], ascending = False, inplace = False)
#strip values with "nan"
df_zip_top_income = df_zip_good[df_zip_good['rank'].notna()]
df_zip_top_income.head()

,zip,lat,lng,rank,place,population,per_cap_inc
473,2108,42.35767,-71.06505,62.0,"Boston, Massachusetts",3446.0,78771.0
474,2109,42.36459,-71.05298,45.0,"Boston, Massachusetts",3428.0,82689.0
475,2110,42.35826,-71.05175,44.0,"Boston, Massachusetts",1428.0,82736.0
524,2199,42.34747,-71.08202,28.0,"Boston, Massachusetts",1005.0,88974.0
579,2493,42.35894,-71.30010,59.0,"Weston, Massachusetts",11469.0,79640.0


In [9]:
#final df to be used that combines all necessary information (i.e. zip cross listed with lng, lat, and per captia)
df_income_zip_good = df_zip_top_income.filter(['zip', 'lat', 'lng', 'rank', 'place', 'population', 'per_cap_inc'], axis =1)
df_income_zip_good = df_income_zip_good.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
df_income_zip_good.columns = ['old_index','zip', 'lat', 'lng', 'rank','place', 'population', 'per_cap_inc']
df_income_zip_good.head()

,old_index,zip,lat,lng,rank,place,population,per_cap_inc
0,473,2108,42.35767,-71.06505,62.0,"Boston, Massachusetts",3446.0,78771.0
1,474,2109,42.36459,-71.05298,45.0,"Boston, Massachusetts",3428.0,82689.0
2,475,2110,42.35826,-71.05175,44.0,"Boston, Massachusetts",1428.0,82736.0
3,524,2199,42.34747,-71.08202,28.0,"Boston, Massachusetts",1005.0,88974.0
4,579,2493,42.35894,-71.30010,59.0,"Weston, Massachusetts",11469.0,79640.0


In [10]:
#check output
#print (df_income_zip_good.head(1000))
print (df_income_zip_good.dtypes)
print (df_income_zip_good.shape)

old_index        int64
zip              int64
lat            float64
lng            float64
rank           float64
place           object
population     float64
per_cap_inc    float64
dtype: object
(95, 8)


In [11]:
#render data set visually into map to display locations
lati = 37.09024
longi =  -95.712891
map_usa = folium.Map(location=[lati, longi], zoom_start = 5)
for i, series in df_income_zip_good.iterrows():
    lat = series ['lat']
    lng = series ['lng']
    town = series ['place']

    folium.Marker (location=[lat,lng], popup = town, icon = folium.Icon(color='blue')).add_to(map_usa)
map_usa

In [12]:
#create placeholder dataframe with HQ and plot onto Map
#geocoordinates for hq to compare with client requirements
datahq = [[41.1489, -73.9830, "Rockland HQ"]]
df_hq_loc = pd.DataFrame(datahq, columns = ['lat', 'lng', 'place'])
#latihq = 41.1489
#longihq = -73.9830
#map_hq = folium.map (location=[latihq,longihq], zoom_start = 5)
#folium.Marker (location=[latiq,longihq], popup = ("HQ"), icon = folium.Icon(color='yellow')).add_to(map_usa)
df_hq_loc

,lat,lng,place
0,41.1489,-73.983,Rockland HQ


In [13]:
map_hq = folium.Map(location=[lati, longi], zoom_start = 5)
for i, series in df_hq_loc.iterrows():
    lat = series ['lat']
    lng = series ['lng']
    town = series ['place']
    folium.Circle(location=[lat,lng], radius = 32186.9, color = 'orange').add_to(map_usa)
#map_usa
for i, series in df_hq_loc.iterrows():
    lat = series ['lat']
    lng = series ['lng']
    town = series ['place']
    folium.Circle(location=[lat,lng], radius =  72420.5, color = 'orange').add_to(map_usa)
map_usa

In [14]:
#following locations meet criteria set forth in reserach problem / question
# pos_loc = possible locations
df_pos_loc = (df_income_zip_good.loc[[12,13,14]])
df_pos_loc.head()

,old_index,zip,lat,lng,rank,place,population,per_cap_inc
12,1991,7021,40.82686,-74.27973,67.0,"Essex Fells, New Jersey",2151.0,77787.0
13,2039,7078,40.74207,-74.33378,21.0,"Short Hills, New Jersey",12849.0,92940.0
14,2135,7608,40.85368,-74.06129,92.0,"Teterboro, New Jersey",18.0,72613.0


In [15]:
# reset index on df
df_pos_loc_good = df_pos_loc.reset_index (level=None, drop=False, col_level=0, col_fill ='')
df_pos_loc_good

,index,old_index,zip,lat,lng,rank,place,population,per_cap_inc
0,12,1991,7021,40.82686,-74.27973,67.0,"Essex Fells, New Jersey",2151.0,77787.0
1,13,2039,7078,40.74207,-74.33378,21.0,"Short Hills, New Jersey",12849.0,92940.0
2,14,2135,7608,40.85368,-74.06129,92.0,"Teterboro, New Jersey",18.0,72613.0


In [16]:
# modify marker lable for map display
df_pos_loc_marker = df_pos_loc_good.filter (['zip', 'rank', 'place', 'per_cap_inc'], axis = 1)
df_pos_loc_marker.head()

,zip,rank,place,per_cap_inc
0,7021,67.0,"Essex Fells, New Jersey",77787.0
1,7078,21.0,"Short Hills, New Jersey",92940.0
2,7608,92.0,"Teterboro, New Jersey",72613.0


In [17]:
#towns selected for final consideration as specified by research question mapped - locations marked in blue
lati = 40.8761622
longi = -74.240092
map_pos_loc = folium.Map(location=[lati, longi], zoom_start = 10)
for i, series in df_pos_loc_good.iterrows():
    lat = series ['lat']
    lng = series ['lng']
    town = series ['place']

    folium.Marker (location=[lat,lng], popup = (town), icon = folium.Icon(color='blue')).add_to(map_pos_loc)
    folium.Circle(location=[lat,lng], radius = 8046.72, color = 'orange').add_to(map_pos_loc)
map_pos_loc

In [18]:
#load competitor data - probable competing businesses in selected locations
url_comp = "https://raw.githubusercontent.com/adhorvitz/IBM_Coursera_Capstone/main/competitors_new.csv"
df_competitors = pd.read_csv(url_comp)
df_competitors.head()

,name,categories,lat,lng,postalcode,city,state,address
0,Fludi Designs,IT Managed Services and Consulting - General,40.706696,-74.302050,7083,Union,NJ,2816 Morris Ave
1,Pinnacle Solutions Incorporated,IT Managed Services and Consulting - Finance,40.716955,-74.331940,7081,Springfield,NJ,"673 Morris Ave, # 1"
2,uBreakiFix,IT break-fix,40.724291,-74.306823,7041,Millburn,NJ,304 Millburn Ave
3,MacWorks360,IT Managed Services and Consulting - General,40.724237,-74.306356,7041,Millburn,NJ,"300 Millburn Ave, #21"
4,International Network & Cabling Services,IT Managed Services and Consulting - General,40.754436,-74.309129,7078,Short Hills,NJ,911 S Orange Ave


In [19]:
#red for competitors - need something to loop and put in only venue name and category in popup
for lat, lng, cat, name in zip (df_competitors['lat'],df_competitors['lng'],df_competitors['categories'], df_competitors['name']):
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        color='black',
        popup= (name, cat),
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(map_pos_loc)
    #display map
map_pos_loc

In [20]:
# @hidden_cell
# specifying credentials for Foursquare
CLIENT_ID = 'YBCSZCXFDWXUBOO33BVHOIVFNMKTNZD3LF3KBIHVTGU0BGIJ'
CLIENT_SECRET = 'GYEF12HHL5DCUC13WWY3JJSIHRMHE5IP1L45HIHQQH0GTQ4V'
VERSION = '20180604'

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    LIMIT = 500
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(type(results))
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    print(venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
   
    return(nearby_venues)

df_venues = getNearbyVenues(names=df_pos_loc_good['place'],
                                   latitudes=df_pos_loc_good['lat'],
                                   longitudes=df_pos_loc_good['lng'])

[[('Essex Fells, New Jersey', 40.82686, -74.27973, 'Cloverleaf Tavern', 40.841311114463146, -74.27856160150759, 'Pub'), ('Essex Fells, New Jersey', 40.82686, -74.27973, "Calandra's Italian Village", 40.836641, -74.273774, 'Italian Restaurant'), ('Essex Fells, New Jersey', 40.82686, -74.27973, 'Pleasantdale Chateau', 40.816265198045876, -74.26657509223364, 'Restaurant'), ('Essex Fells, New Jersey', 40.82686, -74.27973, 'Dough Artisan Pizzeria', 40.84209449999999, -74.28017969999999, 'Italian Restaurant'), ('Essex Fells, New Jersey', 40.82686, -74.27973, "Angeloni's Ristorante & Pizzeria", 40.84005631122611, -74.277269109234, 'Pizza Place'), ('Essex Fells, New Jersey', 40.82686, -74.27973, 'Forte Pizzeria', 40.83636063717328, -74.27178452466734, 'Italian Restaurant'), ('Essex Fells, New Jersey', 40.82686, -74.27973, 'Verona Park', 40.826989240346414, -74.24618044695073, 'Park'), ('Essex Fells, New Jersey', 40.82686, -74.27973, 'Gelotti Ice Cream', 40.83643833428671, -74.27188954158721, '

In [22]:
print (df_venues.shape)
df_venues.head()

(300, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Essex Fells, New Jersey",40.82686,-74.27973,Cloverleaf Tavern,40.841311,-74.278562,Pub
1,"Essex Fells, New Jersey",40.82686,-74.27973,Calandra's Italian Village,40.836641,-74.273774,Italian Restaurant
2,"Essex Fells, New Jersey",40.82686,-74.27973,Pleasantdale Chateau,40.816265,-74.266575,Restaurant
3,"Essex Fells, New Jersey",40.82686,-74.27973,Dough Artisan Pizzeria,40.842094,-74.280180,Italian Restaurant
4,"Essex Fells, New Jersey",40.82686,-74.27973,Angeloni's Ristorante & Pizzeria,40.840056,-74.277269,Pizza Place


In [23]:
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Essex Fells, New Jersey",100,100,100,100,100,100
"Short Hills, New Jersey",100,100,100,100,100,100
"Teterboro, New Jersey",100,100,100,100,100,100


In [24]:
print ("there are {} unique categories.".format(len(df_venues['Venue Category'].unique())))

there are 100 unique categories.


In [25]:
#potential clients in orange
for lat, lng, name, cat in zip (df_venues['Venue Latitude'],df_venues['Venue Longitude'],df_venues['Venue'],df_venues['Venue Category']):
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        color='red',
        popup= (name),
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_pos_loc)
    #display map
map_pos_loc

In [26]:
#one hot encoding - convert categorical into dummy / indicator variable
df_venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_venues_onehot['Neighborhood'] = df_venues['Neighborhood']

#move neighborhood column to first column for ease of inspection
fixed_columns_venues = [df_venues_onehot.columns[-1]] + list(df_venues_onehot.columns[:-1])
df_venues_onehot = df_venues_onehot[fixed_columns_venues]

df_venues_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bar,Baseball Field,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Café,Cajun / Creole Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Football Stadium,French Restaurant,Furniture / Home Store,Garden,Gastropub,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Hot Dog Joint,Ice Cream Shop,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Soba Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,"Essex Fells, New Jersey",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Essex Fells, New Jersey",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Essex Fells, New Jersey",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Essex Fells, New Jersey",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Essex Fells, New Jersey",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df_venues_onehot.shape

(300, 101)

In [28]:
df_venues_grouped = df_venues_onehot.groupby('Neighborhood').mean().reset_index()
df_venues_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bar,Baseball Field,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Café,Cajun / Creole Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Football Stadium,French Restaurant,Furniture / Home Store,Garden,Gastropub,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Hot Dog Joint,Ice Cream Shop,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Soba Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,"Essex Fells, New Jersey",0.03,0.00,0.01,0.00,0.04,0.06,0.02,0.00,0.01,0.01,0.00,0.01,0.02,0.01,0.03,0.04,0.01,0.00,0.00,0.05,0.01,0.00,0.01,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.03,0.00,0.07,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.04,0.00,0.01,0.03,0.00,0.01,0.00,0.01,0.02,0.02,0.02,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.02,0.02,0.01,0.00
1,"Short Hills, New Jersey",0.04,0.00,0.00,0.00,0.03,0.04,0.01,0.01,0.00,0.02,0.01,0.00,0.00,0.01,0.00,0.03,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.01,0.01,0.00,0.02,0.00,0.01,0.03,0.04,0.01,0.00,0.00,0.01,0.01,0.06,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.08,0.01,0.00,0.04,0.02,0.01,0.01,0.00,0.01,0.01,0.03,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.03,0.00,0.01,0.02
2,"Teterboro, New Jersey",0.02,0.01,0.00,0.01,0.01,0.08,0.00,0.01,0.00,0.00,0.00,0.00,0.02,0.04,0.02,0.00,0.00,0.01,0.01,0.02,0.00,0.02,0.02,0.01,0.00,0.02,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.01,0.04,0.04,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.04,0.02,0.01,0.01,0.02,0.05,0.00,0.01,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.06,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.01,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.00


In [29]:
df_venues_grouped.shape

(3, 101)

In [30]:
num_top_venues = 5
for hood in df_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_venues_grouped[df_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Essex Fells, New Jersey----
                venue  freq
0  Italian Restaurant  0.07
1              Bakery  0.06
2         Coffee Shop  0.05
3          Bagel Shop  0.04
4                Park  0.04


----Short Hills, New Jersey----
                venue  freq
0                Park  0.08
1  Italian Restaurant  0.06
2              Bakery  0.04
3                 Gym  0.04
4         Pizza Place  0.04


----Teterboro, New Jersey----
                venue  freq
0              Bakery  0.08
1         Pizza Place  0.06
2  Mexican Restaurant  0.05
3  Italian Restaurant  0.05
4             Brewery  0.04




In [31]:
#reoganize the different categories per towns back into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
#new dataframe to display top 10 venues for each location
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_venues_grouped['Neighborhood']

for ind in np.arange(df_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Essex Fells, New Jersey",Italian Restaurant,Bakery,Coffee Shop,Bagel Shop,Park,Deli / Bodega,Café,American Restaurant,Burger Joint,Ice Cream Shop
1,"Short Hills, New Jersey",Park,Italian Restaurant,Bakery,Gym,Pizza Place,American Restaurant,Bagel Shop,Coffee Shop,Sandwich Place,Wine Shop
2,"Teterboro, New Jersey",Bakery,Pizza Place,Mexican Restaurant,Italian Restaurant,Brewery,Korean Restaurant,Ice Cream Shop,Hot Dog Joint,Park,Cosmetics Shop


Cluserting - k-means, unsupervised machine learning for (categorical) data

In [33]:
# set number of clusters
kclusters = 3

df_venues_grouped_clustering = df_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1])

In [34]:
df_venues_grouped_clustering

,American Restaurant,Argentinian Restaurant,Art Museum,Asian Restaurant,Bagel Shop,Bakery,Bar,Baseball Field,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Café,Cajun / Creole Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Football Stadium,French Restaurant,Furniture / Home Store,Garden,Gastropub,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Hot Dog Joint,Ice Cream Shop,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Soba Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,0.03,0.00,0.01,0.00,0.04,0.06,0.02,0.00,0.01,0.01,0.00,0.01,0.02,0.01,0.03,0.04,0.01,0.00,0.00,0.05,0.01,0.00,0.01,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.03,0.00,0.07,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.04,0.00,0.01,0.03,0.00,0.01,0.00,0.01,0.02,0.02,0.02,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.02,0.02,0.01,0.00
1,0.04,0.00,0.00,0.00,0.03,0.04,0.01,0.01,0.00,0.02,0.01,0.00,0.00,0.01,0.00,0.03,0.00,0.00,0.00,0.03,0.00,0.00,0.01,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.01,0.01,0.00,0.02,0.00,0.01,0.03,0.04,0.01,0.00,0.00,0.01,0.01,0.06,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.08,0.01,0.00,0.04,0.02,0.01,0.01,0.00,0.01,0.01,0.03,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.03,0.00,0.01,0.02
2,0.02,0.01,0.00,0.01,0.01,0.08,0.00,0.01,0.00,0.00,0.00,0.00,0.02,0.04,0.02,0.00,0.00,0.01,0.01,0.02,0.00,0.02,0.02,0.01,0.00,0.02,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.01,0.04,0.04,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.04,0.02,0.01,0.01,0.02,0.05,0.00,0.01,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.06,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.01,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.00


In [35]:
df_pos_loc_good

,index,old_index,zip,lat,lng,rank,place,population,per_cap_inc
0,12,1991,7021,40.82686,-74.27973,67.0,"Essex Fells, New Jersey",2151.0,77787.0
1,13,2039,7078,40.74207,-74.33378,21.0,"Short Hills, New Jersey",12849.0,92940.0
2,14,2135,7608,40.85368,-74.06129,92.0,"Teterboro, New Jersey",18.0,72613.0


In [36]:
df_pos_loc_good_stripped = df_pos_loc_good.drop (['index', 'old_index'], 1)
df_pos_loc_good_stripped

,zip,lat,lng,rank,place,population,per_cap_inc
0,7021,40.82686,-74.27973,67.0,"Essex Fells, New Jersey",2151.0,77787.0
1,7078,40.74207,-74.33378,21.0,"Short Hills, New Jersey",12849.0,92940.0
2,7608,40.85368,-74.06129,92.0,"Teterboro, New Jersey",18.0,72613.0


In [37]:
df_pos_loc_good_reorg = df_pos_loc_good_stripped [['place', 'lat', 'lng', 'zip', 'population', 'per_cap_inc', 'rank']]

In [38]:
df_pos_loc_good_reorg = df_pos_loc_good_reorg.rename(columns = {"place": "Neighborhood"})
df_pos_loc_good_reorg

,Neighborhood,lat,lng,zip,population,per_cap_inc,rank
0,"Essex Fells, New Jersey",40.82686,-74.27973,7021,2151.0,77787.0,67.0
1,"Short Hills, New Jersey",40.74207,-74.33378,7078,12849.0,92940.0,21.0
2,"Teterboro, New Jersey",40.85368,-74.06129,7608,18.0,72613.0,92.0


In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Essex Fells, New Jersey",Italian Restaurant,Bakery,Coffee Shop,Bagel Shop,Park,Deli / Bodega,Café,American Restaurant,Burger Joint,Ice Cream Shop
1,2,"Short Hills, New Jersey",Park,Italian Restaurant,Bakery,Gym,Pizza Place,American Restaurant,Bagel Shop,Coffee Shop,Sandwich Place,Wine Shop
2,1,"Teterboro, New Jersey",Bakery,Pizza Place,Mexican Restaurant,Italian Restaurant,Brewery,Korean Restaurant,Ice Cream Shop,Hot Dog Joint,Park,Cosmetics Shop


In [40]:
df_venues_grouped_merged = df_pos_loc_good_reorg

# merge df_venues_grouped_grouped with df_pos_loc_good_data to add latitude/longitude for each neighborhood
df_venues_grouped_merged = df_venues_grouped_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_venues_grouped_merged.head() # check the last columns!

,Neighborhood,lat,lng,zip,population,per_cap_inc,rank,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Essex Fells, New Jersey",40.82686,-74.27973,7021,2151.0,77787.0,67.0,0,Italian Restaurant,Bakery,Coffee Shop,Bagel Shop,Park,Deli / Bodega,Café,American Restaurant,Burger Joint,Ice Cream Shop
1,"Short Hills, New Jersey",40.74207,-74.33378,7078,12849.0,92940.0,21.0,2,Park,Italian Restaurant,Bakery,Gym,Pizza Place,American Restaurant,Bagel Shop,Coffee Shop,Sandwich Place,Wine Shop
2,"Teterboro, New Jersey",40.85368,-74.06129,7608,18.0,72613.0,92.0,1,Bakery,Pizza Place,Mexican Restaurant,Italian Restaurant,Brewery,Korean Restaurant,Ice Cream Shop,Hot Dog Joint,Park,Cosmetics Shop


In [41]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(df_venues_grouped_merged['lat'], df_venues_grouped_merged['lng'], df_venues_grouped_merged['Neighborhood'], df_venues_grouped_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_pos_loc)
       
map_pos_loc